In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Load datasets
train_df = pd.read_csv('DiabeticRetinopathy_train.csv', header=None)
val_df = pd.read_csv('DiabeticRetinopathy_validation.csv', header=None)
test_df = pd.read_csv('DiabeticRetinopathy_test.csv', header=None)

# Prepare data
def prepare_data(df):
    X = df.iloc[:, :-1].values  # All columns except last
    y = df.iloc[:, -1].values.reshape(-1, 1)  # Last column as target
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

X_train, y_train = prepare_data(train_df)
X_val, y_val = prepare_data(val_df)
X_test, y_test = prepare_data(test_df)

# Create DataLoaders
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Neural Network Architecture
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_layers):
        super().__init__()
        layers = []
        in_features = input_size
        
        # Add hidden layers
        for idx, hidden_size in enumerate(hidden_layers):
            layers.append(nn.Linear(in_features, hidden_size))
            layers.append(nn.Sigmoid())
            in_features = hidden_size  # Update input size for next layer
            
        # Output layer
        layers.append(nn.Linear(in_features, 1))
        layers.append(nn.Sigmoid())
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

# Training Function
def train_model(model, criterion, optimizer, epochs=100):
    train_losses = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
        # Store average loss per sample
        train_losses.append(epoch_loss / len(train_dataset))
        
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_losses[-1]:.4f}')
    
    return train_losses

# Evaluation Function
def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        predicted = (outputs >= 0.5).float()
        y_np = y.numpy().flatten()
        pred_np = predicted.numpy().flatten()
        
        return {
            'confusion_matrix': confusion_matrix(y_np, pred_np),
            'accuracy': accuracy_score(y_np, pred_np),
            'precision': precision_score(y_np, pred_np),
            'recall': recall_score(y_np, pred_np),
            'f1': f1_score(y_np, pred_np)
        }

# Experiment Configuration
architectures = [
    # Single hidden layer architectures
    {'name': '10-neuron', 'layers': [10]},
    {'name': '100-neuron', 'layers': [100]},
    # Add more architectures as needed
]

# Training and Evaluation
for arch in architectures:
    print(f"\n=== Training {arch['name']} Architecture ===")
    
    # Initialize model
    model = NeuralNet(input_size=19, hidden_layers=arch['layers'])
    criterion = nn.MSELoss(reduction='sum')  # SSE Loss
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    # Train model
    train_losses = train_model(model, criterion, optimizer, epochs=100)
    
    # Evaluate
    val_metrics = evaluate_model(model, X_val, y_val)
    test_metrics = evaluate_model(model, X_test, y_test)
    
    # Print results
    print(f"\nValidation Metrics ({arch['name']}):")
    print(f"Accuracy: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1']:.4f}")
    print(f"Test Metrics ({arch['name']}):")
    print(f"Accuracy: {test_metrics['accuracy']:.4f}, F1: {test_metrics['f1']:.4f}")
